In [1]:
import torch
import torch.nn as nn

In [2]:
inputs = torch.tensor([
    [0.72, 0.45, 0.31],
    [0.75, 0.20, 0.55],
    [0.30, 0.80, 0.40],
    [0.85, 0.35, 0.60],
    [0.55, 0.15, 0.75],
    [0.25, 0.20, 0.85]
])

In [3]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape)

torch.Size([2, 6, 3])


In [4]:
class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, _ = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2)
        attn_scores.masked_fill_(
            self.mask.bool()[:num_tokens, :num_tokens],
            -torch.inf
        )

        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5,
            dim=-1
        )

        attn_weights = self.dropout(attn_weights)
        context_vec = attn_weights @ values
        return context_vec


In [5]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [
                CausalAttention(d_in, d_out, context_length, dropout, qkv_bias)
                for _ in range(num_heads)
            ]
        )

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)


In [6]:
torch.manual_seed(123)

context_length = batch.shape[1]
d_in, d_out = 3, 2

mha = MultiHeadAttentionWrapper(
    d_in=d_in,
    d_out=d_out,
    context_length=context_length,
    dropout=0.0,
    num_heads=2
)

In [7]:
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.5762, -0.1627,  0.5569,  0.3635],
         [-0.5650, -0.0630,  0.5599,  0.3006],
         [-0.5472, -0.1226,  0.5285,  0.3435],
         [-0.5787, -0.0943,  0.5621,  0.3388],
         [-0.5593, -0.0436,  0.5509,  0.3046],
         [-0.5287, -0.0033,  0.5277,  0.2743]],

        [[-0.5762, -0.1627,  0.5569,  0.3635],
         [-0.5650, -0.0630,  0.5599,  0.3006],
         [-0.5472, -0.1226,  0.5285,  0.3435],
         [-0.5787, -0.0943,  0.5621,  0.3388],
         [-0.5593, -0.0436,  0.5509,  0.3046],
         [-0.5287, -0.0033,  0.5277,  0.2743]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])
